In [6]:
!pip install -qq transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00


# Build starcoder.cpp project

In [1]:
!git clone https://github.com/bigcode-project/starcoder.cpp

Cloning into 'starcoder.cpp'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 103 (delta 30), reused 91 (delta 24), pack-reused 0
Receiving objects: 100% (103/103), 7.28 MiB | 18.49 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [8]:
!cd starcoder.cpp && make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native
I LDFLAGS:  
I CC:       cc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
I CXX:      g++ (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0

g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -c common.cpp -o common.o
cc  -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -std=

## Download Model and convert it to GGML format

In [7]:
!python starcoder.cpp/convert-hf-to-ggml.py rahuldshetty/tiny-starcoder-instruct

2023-07-22 11:26:25.629875: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading model:  rahuldshetty/tiny-starcoder-instruct
Model loaded:  rahuldshetty/tiny-starcoder-instruct
{'vocab_size': 49152, 'n_positions': 8192, 'n_embd': 768, 'n_layer': 20, 'n_head': 12, 'n_inner': 3072, 'activation_function': 'gelu_pytorch_tanh', 'resid_pdrop': 0.1, 'embd_pdrop': 0.1, 'attn_pdrop': 0.1, 'layer_norm_epsilon': 1e-05, 'initializer_range': 0.02, 'scale_attn_weights': True, 'use_cache': False, 'attention_softmax_in_fp32': True, 'scale_attention_softmax_in_fp32': True, 'multi_query': True, 'bos_token_id': 0, 'eos_token_id': 0, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': 'float32', 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_att

In [9]:
!cd starcoder.cpp && ./quantize --help

usage: ./quantize model-f32.bin model-quant.bin type
  type = "q4_0" or 2
  type = "q4_1" or 3
  type = "q5_0" or 8
  type = "q5_1" or 9
  type = "q8_0" or 7


In [12]:
!cd starcoder.cpp && ./quantize ../models/rahuldshetty/tiny-starcoder-instruct-ggml.bin ../quantized/tiny-starcoder-instruct-ggml-q4_0.bin 2

starcoder_model_quantize: loading model from '../models/rahuldshetty/tiny-starcoder-instruct-ggml.bin'
starcoder_model_quantize: n_vocab     = 49152
starcoder_model_quantize: n_ctx       = 8192
starcoder_model_quantize: n_embd      = 768
starcoder_model_quantize: n_head      = 12
starcoder_model_quantize: n_layer     = 20
starcoder_model_quantize: ftype (src) = 1
starcoder_model_quantize: qntvr (src) = 0
starcoder_model_quantize: ftype (dst) = 1002
starcoder_model_quantize: qntvr (dst) = 1
                                                       model/wte - [  768, 49152,     1], type =    f16 size =   144.00 MB ->    22.50 MB | hist: 0.036 0.013 0.021 0.033 0.050 0.073 0.099 0.123 0.132 0.123 0.100 0.074 0.051 0.034 0.021 0.018 
                                                       model/wpe - [  768,  8192,     1], type =    f32 size =   24.000 MB
                                                 model/h0/ln_1/g - [  768,     1,     1], type =    f32 size =    0.003 MB
                

In [13]:
!cd starcoder.cpp && ./quantize ../models/rahuldshetty/tiny-starcoder-instruct-ggml.bin ../quantized/tiny-starcoder-instruct-ggml-q5_0.bin 8

starcoder_model_quantize: loading model from '../models/rahuldshetty/tiny-starcoder-instruct-ggml.bin'
starcoder_model_quantize: n_vocab     = 49152
starcoder_model_quantize: n_ctx       = 8192
starcoder_model_quantize: n_embd      = 768
starcoder_model_quantize: n_head      = 12
starcoder_model_quantize: n_layer     = 20
starcoder_model_quantize: ftype (src) = 1
starcoder_model_quantize: qntvr (src) = 0
starcoder_model_quantize: ftype (dst) = 1008
starcoder_model_quantize: qntvr (dst) = 1
                                                       model/wte - [  768, 49152,     1], type =    f16 size =   144.00 MB ->    24.75 MB | hist: 0.083 0.064 0.057 0.053 0.052 0.058 0.066 0.075 0.087 0.067 0.059 0.052 0.050 0.052 0.059 0.066 
                                                       model/wpe - [  768,  8192,     1], type =    f32 size =   24.000 MB
                                                 model/h0/ln_1/g - [  768,     1,     1], type =    f32 size =    0.003 MB
                

In [14]:
!cd starcoder.cpp && ./quantize ../models/rahuldshetty/tiny-starcoder-instruct-ggml.bin ../quantized/tiny-starcoder-instruct-ggml-q8_0.bin 7

starcoder_model_quantize: loading model from '../models/rahuldshetty/tiny-starcoder-instruct-ggml.bin'
starcoder_model_quantize: n_vocab     = 49152
starcoder_model_quantize: n_ctx       = 8192
starcoder_model_quantize: n_embd      = 768
starcoder_model_quantize: n_head      = 12
starcoder_model_quantize: n_layer     = 20
starcoder_model_quantize: ftype (src) = 1
starcoder_model_quantize: qntvr (src) = 0
starcoder_model_quantize: ftype (dst) = 1007
starcoder_model_quantize: qntvr (dst) = 1
                                                       model/wte - [  768, 49152,     1], type =    f16 size =   144.00 MB ->    40.50 MB | hist: 0.000 0.026 0.017 0.027 0.042 0.063 0.088 0.114 0.254 0.113 0.087 0.061 0.041 0.026 0.016 0.025 
                                                       model/wpe - [  768,  8192,     1], type =    f32 size =   24.000 MB
                                                 model/h0/ln_1/g - [  768,     1,     1], type =    f32 size =    0.003 MB
                